In [1]:
from selenium import webdriver
from tqdm.notebook import tqdm
import pandas as pd
import json
from time import sleep
import os
from IPython.display import clear_output


In [4]:
driver = webdriver.PhantomJS(r'../phantomjs.exe')
def read_config(config_path):
    with open(config_path, 'r', encoding='UTF-8') as config_file:
        return json.load(config_file)
config = read_config(r'../config.json')
base_url = 'https://www.gismeteo.ru/diary/{}/{}/{}/'
years = range(2018, 2021)
months = range(1, 13)
def get_img(el):
    _ = el.find_elements_by_tag_name('img')
    if len(_) > 0:
        return os.path.basename(_[0].get_attribute('src'))[:-4]
    return '-'

def parse_wind(el):
    text = el.text
    _ = text.split(' ')
    if len(_) > 1:
        direction = _[0]
        wind = _[1]
        return [direction, int(''.join(filter(str.isdigit, wind)))]
    return ['-', 0]

In [5]:
result = []
for areal in tqdm(config['urls']):
    for city in config['urls'][areal]:
        url = config['urls'][areal][city]
        id = url.split('-')[-1][:-1]
        for year in years:
            for month in months:
                url_ = base_url.format(id, year, month)
                driver.get(url_)
                table = driver.find_element_by_css_selector('table')
                for row in table.find_elements_by_css_selector('tr'):
                    cols = row.find_elements_by_tag_name('td')
                    if len(cols) == 0:
                        continue
                    try:
                        data = {
                            'day_temp': int(cols[1].text),
                            'day_pressure': int(cols[2].text),
                            'day_obl': get_img(cols[3]),
                            'day_phen': get_img(cols[4]),
                            'day_dir': parse_wind(cols[5])[0],
                            'day_wind': parse_wind(cols[5])[1],
#                             'night_temp': int(cols[6].text),
#                             'night_pressure': int(cols[7].text),
#                             'night_obl': get_img(cols[8]),
#                             'night_phen': get_img(cols[9]),
#                             'night_dir': parse_wind(cols[10])[0],
#                             'night_wind': parse_wind(cols[10])[1],
                        }
                        data.update({
                            'areal': areal,
                            'city': city,
                            'year': year,
                            'month': month,
                            'day': int(cols[0].text)
                        })
                        result.append(data)
                    except Exception as e:
                        print(e)

dt = pd.DataFrame(result)
dt.to_csv('result.csv', index=False)

In [140]:
train_data = []
for idx, row in tqdm(dt.iterrows()):
    if row['щука'] > -1:
        data = dt[idx - 3: idx + 1]
        temps = list(data['day_temp'])
        press = list(data['day_pressure'])
        obls = list(data['day_obl'])
        phens = list(data['day_phen'])
        dirs = list(data['day_dir'])
        winds = list(data['day_wind'])
        areal = row['areal']
        month = row['month']
        day = row['day']
        for fish in fishs:
            row_ = {'areal': areal, 'month': month, 'day': day, 'fish': fish, 'forecast': row[fish]}
            for k in range(len(temps)):
                row_.update({
                    'temperature_{}'.format(k): temps[3 - k],
                    'pressure_{}'.format(k): press[3 - k],
                    'obl_{}'.format(k): obls[3 - k],
                    'phenomen_{}'.format(k): phens[3 - k],
                    'wind_direction_{}'.format(k): dirs[3 - k],
                    'winds_{}'.format(k): winds[3 - k],
                })
            train_data.append(row_)

In [176]:
train_dt = pd.DataFrame(train_data)

In [177]:
train_dt.describe()

,month,day,forecast,temperature_0,pressure_0,winds_0,temperature_1,pressure_1,winds_1,temperature_2,pressure_2,winds_2,temperature_3,pressure_3,winds_3
count,1537.000000,1537.000000,1536.000000,1537.00000,1537.000000,1537.000000,1537.000000,1537.000000,1537.000000,1537.000000,1537.000000,1537.000000,1537.000000,1537.000000,1537.000000
mean,6.698113,15.962264,0.405599,11.09434,743.811321,3.603774,9.867925,744.113208,3.018868,9.471698,743.698113,2.962264,10.000000,743.245283,3.037736
std,3.340870,8.655359,0.491167,12.55175,26.023018,2.521225,12.863420,25.951531,2.359938,12.727027,26.339257,2.110522,13.457301,26.749885,2.387551
min,1.000000,1.000000,0.000000,-17.00000,628.000000,0.000000,-18.000000,627.000000,0.000000,-17.000000,626.000000,0.000000,-20.000000,625.000000,0.000000
25%,4.000000,9.000000,0.000000,1.00000,743.000000,2.000000,1.000000,742.000000,1.000000,1.000000,742.000000,2.000000,1.000000,743.000000,2.000000
50%,7.000000,16.000000,0.000000,14.00000,749.000000,3.000000,11.000000,749.000000,2.000000,8.000000,749.000000,3.000000,8.000000,748.000000,2.000000
75%,9.000000,23.000000,1.000000,22.00000,756.000000,5.000000,20.000000,757.000000,4.000000,21.000000,756.000000,3.000000,21.000000,756.000000,4.000000
max,12.000000,30.000000,1.000000,32.00000,776.000000,11.000000,30.000000,781.000000,12.000000,31.000000,782.000000,11.000000,35.000000,779.000000,11.000000


In [178]:
train_dt

,areal,month,day,fish,forecast,temperature_0,pressure_0,obl_0,phenomen_0,wind_direction_0,...,obl_2,phenomen_2,wind_direction_2,winds_2,temperature_3,pressure_3,obl_3,phenomen_3,wind_direction_3,winds_3
0,Алтайский край,5,22,щука,1.0,17,751,dull,storm,З,...,dull,-,З,2,22,741,dull,storm,-,0
1,Алтайский край,5,22,судак,0.0,17,751,dull,storm,З,...,dull,-,З,2,22,741,dull,storm,-,0
2,Алтайский край,5,22,окунь,1.0,17,751,dull,storm,З,...,dull,-,З,2,22,741,dull,storm,-,0
3,Алтайский край,5,22,берш,0.0,17,751,dull,storm,З,...,dull,-,З,2,22,741,dull,storm,-,0
4,Алтайский край,5,22,речная форель,1.0,17,751,dull,storm,З,...,dull,-,З,2,22,741,dull,storm,-,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1532,Московская область,6,24,сазан,1.0,24,748,-,-,СЗ,...,-,-,Ю,1,19,746,-,-,В,2
1533,Московская область,6,24,подуст,1.0,24,748,-,-,СЗ,...,-,-,Ю,1,19,746,-,-,В,2
1534,Московская область,6,24,толстолобик,1.0,24,748,-,-,СЗ,...,-,-,Ю,1,19,746,-,-,В,2
1535,Московская область,6,24,вобла,1.0,24,748,-,-,СЗ,...,-,-,Ю,1,19,746,-,-,В,2


In [179]:
min_ = {
    'temperature_0': -60, 'temperature_1': -60, 'temperature_2': -60, 'temperature_3': -60,
    'pressure_0': 500, 'pressure_1': 500, 'pressure_2': 500, 'pressure_3': 500,
    'winds_0': 0, 'winds_1': 0, 'winds_2': 0, 'winds_3': 0, 'day': 1
}

max_ = {
    'temperature_0': 50, 'temperature_1': 50, 'temperature_2': 50, 'temperature_3': 50,
    'pressure_0': 900, 'pressure_1': 900, 'pressure_2': 900, 'pressure_3': 900,
    'winds_0': 20, 'winds_1': 20, 'winds_2': 20, 'winds_3': 20, 'day': 31
}

In [180]:
for key in min_:
    train_dt[key] = (train_dt[key] - min_[key])/(max_[key] - min_[key])

In [181]:
train_dt.describe()

,month,day,forecast,temperature_0,pressure_0,winds_0,temperature_1,pressure_1,winds_1,temperature_2,pressure_2,winds_2,temperature_3,pressure_3,winds_3
count,1537.000000,1537.000000,1536.000000,1537.000000,1537.000000,1537.000000,1537.000000,1537.000000,1537.000000,1537.000000,1537.000000,1537.000000,1537.000000,1537.000000,1537.000000
mean,6.698113,0.498742,0.405599,0.646312,0.609528,0.180189,0.635163,0.610283,0.150943,0.631561,0.609245,0.148113,0.636364,0.608113,0.151887
std,3.340870,0.288512,0.491167,0.114107,0.065058,0.126061,0.116940,0.064879,0.117997,0.115700,0.065848,0.105526,0.122339,0.066875,0.119378
min,1.000000,0.000000,0.000000,0.390909,0.320000,0.000000,0.381818,0.317500,0.000000,0.390909,0.315000,0.000000,0.363636,0.312500,0.000000
25%,4.000000,0.266667,0.000000,0.554545,0.607500,0.100000,0.554545,0.605000,0.050000,0.554545,0.605000,0.100000,0.554545,0.607500,0.100000
50%,7.000000,0.500000,0.000000,0.672727,0.622500,0.150000,0.645455,0.622500,0.100000,0.618182,0.622500,0.150000,0.618182,0.620000,0.100000
75%,9.000000,0.733333,1.000000,0.745455,0.640000,0.250000,0.727273,0.642500,0.200000,0.736364,0.640000,0.150000,0.736364,0.640000,0.200000
max,12.000000,0.966667,1.000000,0.836364,0.690000,0.550000,0.818182,0.702500,0.600000,0.827273,0.705000,0.550000,0.863636,0.697500,0.550000


In [182]:
train_dt

,areal,month,day,fish,forecast,temperature_0,pressure_0,obl_0,phenomen_0,wind_direction_0,...,obl_2,phenomen_2,wind_direction_2,winds_2,temperature_3,pressure_3,obl_3,phenomen_3,wind_direction_3,winds_3
0,Алтайский край,5,0.700000,щука,1.0,0.700000,0.6275,dull,storm,З,...,dull,-,З,0.10,0.745455,0.6025,dull,storm,-,0.0
1,Алтайский край,5,0.700000,судак,0.0,0.700000,0.6275,dull,storm,З,...,dull,-,З,0.10,0.745455,0.6025,dull,storm,-,0.0
2,Алтайский край,5,0.700000,окунь,1.0,0.700000,0.6275,dull,storm,З,...,dull,-,З,0.10,0.745455,0.6025,dull,storm,-,0.0
3,Алтайский край,5,0.700000,берш,0.0,0.700000,0.6275,dull,storm,З,...,dull,-,З,0.10,0.745455,0.6025,dull,storm,-,0.0
4,Алтайский край,5,0.700000,речная форель,1.0,0.700000,0.6275,dull,storm,З,...,dull,-,З,0.10,0.745455,0.6025,dull,storm,-,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1532,Московская область,6,0.766667,сазан,1.0,0.763636,0.6200,-,-,СЗ,...,-,-,Ю,0.05,0.718182,0.6150,-,-,В,0.1
1533,Московская область,6,0.766667,подуст,1.0,0.763636,0.6200,-,-,СЗ,...,-,-,Ю,0.05,0.718182,0.6150,-,-,В,0.1
1534,Московская область,6,0.766667,толстолобик,1.0,0.763636,0.6200,-,-,СЗ,...,-,-,Ю,0.05,0.718182,0.6150,-,-,В,0.1
1535,Московская область,6,0.766667,вобла,1.0,0.763636,0.6200,-,-,СЗ,...,-,-,Ю,0.05,0.718182,0.6150,-,-,В,0.1


In [183]:
def one_hot(df, cols):
    """
    @param df pandas DataFrame
    @param cols a list of columns to encode 
    @return a DataFrame with one-hot encoding
    """
    for each in cols:
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
        df = pd.concat([df, dummies], axis=1)
        df = df.drop(each,axis = 1)
    return df

oh_cols = ['month', 'fish', 'obl_0', 'obl_1', 'obl_2', 'obl_3', 'phenomen_0', 'phenomen_1', 'phenomen_2', 'phenomen_3',
          'wind_direction_0', 'wind_direction_1', 'wind_direction_2', 'wind_direction_3']
train_dt = one_hot(train_dt, oh_cols)

In [184]:
train_dt

,areal,day,forecast,temperature_0,pressure_0,winds_0,temperature_1,pressure_1,winds_1,temperature_2,...,wind_direction_2_ЮЗ,wind_direction_3_-,wind_direction_3_В,wind_direction_3_З,wind_direction_3_С,wind_direction_3_СВ,wind_direction_3_СЗ,wind_direction_3_Ю,wind_direction_3_ЮВ,wind_direction_3_ЮЗ
0,Алтайский край,0.700000,1.0,0.700000,0.6275,0.05,0.681818,0.6225,0.05,0.763636,...,0,1,0,0,0,0,0,0,0,0
1,Алтайский край,0.700000,0.0,0.700000,0.6275,0.05,0.681818,0.6225,0.05,0.763636,...,0,1,0,0,0,0,0,0,0,0
2,Алтайский край,0.700000,1.0,0.700000,0.6275,0.05,0.681818,0.6225,0.05,0.763636,...,0,1,0,0,0,0,0,0,0,0
3,Алтайский край,0.700000,0.0,0.700000,0.6275,0.05,0.681818,0.6225,0.05,0.763636,...,0,1,0,0,0,0,0,0,0,0
4,Алтайский край,0.700000,1.0,0.700000,0.6275,0.05,0.681818,0.6225,0.05,0.763636,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1532,Московская область,0.766667,1.0,0.763636,0.6200,0.15,0.736364,0.6100,0.05,0.754545,...,0,0,1,0,0,0,0,0,0,0
1533,Московская область,0.766667,1.0,0.763636,0.6200,0.15,0.736364,0.6100,0.05,0.754545,...,0,0,1,0,0,0,0,0,0,0
1534,Московская область,0.766667,1.0,0.763636,0.6200,0.15,0.736364,0.6100,0.05,0.754545,...,0,0,1,0,0,0,0,0,0,0
1535,Московская область,0.766667,1.0,0.763636,0.6200,0.15,0.736364,0.6100,0.05,0.754545,...,0,0,1,0,0,0,0,0,0,0


In [185]:
del train_dt['areal']

In [187]:
y = train_dt['forecast'].values.astype('int32')
del train_dt['forecast']
X = train_dt.values

In [188]:
y

array([1, 0, 1, ..., 1, 1, 1])

In [189]:
X

array([[0.7       , 0.7       , 0.6275    , ..., 0.        , 0.        ,
        0.        ],
       [0.7       , 0.7       , 0.6275    , ..., 0.        , 0.        ,
        0.        ],
       [0.7       , 0.7       , 0.6275    , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.76666667, 0.76363636, 0.62      , ..., 0.        , 0.        ,
        0.        ],
       [0.76666667, 0.76363636, 0.62      , ..., 0.        , 0.        ,
        0.        ],
       [0.76666667, 0.76363636, 0.62      , ..., 0.        , 0.        ,
        0.        ]])

In [190]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)


In [191]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

In [192]:
model.fit(X_train, y_train)

C:\Users\Xiaomi\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [193]:
preds = model.predict(X_test)

In [194]:
from sklearn.metrics import accuracy_score

print(accuracy_score(preds, y_test))

0.8409090909090909


In [195]:
model.predict_proba(X_test)

array([[0.10883776, 0.89116224],
       [0.84442879, 0.15557121],
       [0.25171191, 0.74828809],
       [0.98210371, 0.01789629],
       [0.97052611, 0.02947389],
       [0.5179977 , 0.4820023 ],
       [0.87316492, 0.12683508],
       [0.87118667, 0.12881333],
       [0.3715776 , 0.6284224 ],
       [0.72183933, 0.27816067],
       [0.90005368, 0.09994632],
       [0.64236231, 0.35763769],
       [0.17037106, 0.82962894],
       [0.51969555, 0.48030445],
       [0.96188651, 0.03811349],
       [0.2578033 , 0.7421967 ],
       [0.77357354, 0.22642646],
       [0.84116862, 0.15883138],
       [0.88499736, 0.11500264],
       [0.77256384, 0.22743616],
       [0.27599988, 0.72400012],
       [0.83116402, 0.16883598],
       [0.05442305, 0.94557695],
       [0.95930368, 0.04069632],
       [0.16569008, 0.83430992],
       [0.93635407, 0.06364593],
       [0.86424017, 0.13575983],
       [0.82574914, 0.17425086],
       [0.21762555, 0.78237445],
       [0.48328095, 0.51671905],
       [0.